
# **Alpha Earth – Implementación directa con KML/KMZ (Cafetales Chiapas)**
**API:** Google Earth Engine (Python) + `geemap` + `leafmap`  
**Zonas:** `Parcelas Triunfo Verde Chiapas.kml`, `Poligono parcela Oscar.kmz`  

> Delimitar zonas (KML/KMZ) de cafetales y calcular **NDVI/NDWI** con Sentinel‑2 SR (máscara QA60), **Embeddings (Annual, 64D)** con **PCA RGB**, **Similaridad** y **Cambio YoY**. Incluye **estadísticos zonales** y exportes opcionales.


In [8]:

# === Dependencias ===
import sys

def _pip(pkg):
    try:
        from IPython import get_ipython
        ipy = get_ipython()
        if ipy is not None:
            ipy.system(f"{sys.executable} -m pip install --quiet --upgrade {pkg}")
        else:
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "--upgrade", pkg])
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "--upgrade", pkg])

for p in ["earthengine-api", "geemap>=0.32.0", "leafmap>=0.34.0", "fastkml", "shapely", "pandas"]:
    _pip(p)

print("✅ Dependencias listas. Reinicia el kernel y vuelve a ejecutar las celdas.")


✅ Dependencias listas. Reinicia el kernel y vuelve a ejecutar las celdas.


In [ ]:

%pip install earthengine-api --quiet

# === Autenticación e inicialización de Earth Engine ===
import ee
try:
    ee.Initialize()
    print("✅ EE inicializado (token encontrado).")
except Exception:
    print("ℹ️ Autenticación requerida. Se abrirá el flujo OAuth.")
    ee.Authenticate(auth_mode="notebook")
    ee.Initialize()
    print("✅ EE autenticado e inicializado.")


Note: you may need to restart the kernel to use updated packages.
ℹ️ Autenticación requerida. Se abrirá el flujo OAuth.


In [ ]:
import sys

def _pip(pkg):
    try:
        from IPython import get_ipython
        ipy = get_ipython()
        if ipy is not None:
            ipy.system(f"{sys.executable} -m pip install --quiet --upgrade {pkg}")
        else:
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "--upgrade", pkg])
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "--upgrade", pkg])

for p in ["earthengine-api", "geemap>=0.32.0", "leafmap>=0.34.0", "fastkml", "shapely", "pandas"]:
    _pip(p)

print("✅ Dependencias listas. Reinicia el kernel y vuelve a ejecutar las celdas.")

In [ ]:

# === Imports, rutas y parámetros ===
import ee, json, zipfile
import leafmap, geemap
import pandas as pd
from fastkml import kml
from shapely.geometry import mapping

KML_LOCAL_PATH = "/mnt/data/Parcelas Triunfo Verde Chiapas.kml"
KMZ_LOCAL_PATH = "/mnt/data/Poligono parcela Oscar.kmz"

YEAR = 2025
DATE_START = ee.Date.fromYMD(YEAR, 7, 1)
DATE_END   = ee.Date.fromYMD(YEAR, 7, 31)
CLOUDY_PCT = 30
ZONE_KEY   = "zone_id"


In [ ]:

# === Utilidades: KML/KMZ locales -> ee.FeatureCollection ===
def _kml_to_geojson_dict(kml_bytes: bytes) -> dict:
    k = kml.KML()
    k.from_string(kml_bytes)

    def _gather(kobj):
        feats = []
        for d in kobj.features():
            if hasattr(d, "features"):
                feats.extend(_gather(d))
            else:
                feats.append(d)
        return feats

    feats = _gather(k)
    fc = {"type": "FeatureCollection", "features": []}
    for f in feats:
        if f.geometry is None:
            continue
        props = {}
        if getattr(f, "name", None):
            props["name"] = f.name
        if getattr(f, "description", None):
            props["description"] = f.description
        try:
            if f.extended_data and f.extended_data.elements:
                for elem in f.extended_data.elements:
                    if hasattr(elem, "data"):
                        for d in elem.data:
                            key = getattr(d, "name", None)
                            val = getattr(d, "value", None)
                            if key:
                                props[key] = val
        except Exception:
            pass
        fc["features"].append({
            "type": "Feature",
            "geometry": mapping(f.geometry),
            "properties": props
        })
    return fc

def ee_fc_from_local_kml(path: str) -> ee.FeatureCollection:
    with open(path, "rb") as fp:
        kml_bytes = fp.read()
    gj = _kml_to_geojson_dict(kml_bytes)
    return geemap.geojson_to_ee(gj)

def ee_fc_from_local_kmz(path: str) -> ee.FeatureCollection:
    with zipfile.ZipFile(path, "r") as zf:
        kml_names = [n for n in zf.namelist() if n.lower().endswith(".kml")]
        if not kml_names:
            raise ValueError("KMZ no contiene un archivo .kml")
        with zf.open(kml_names[0], "r") as fp:
            kml_bytes = fp.read()
    gj = _kml_to_geojson_dict(kml_bytes)
    return geemap.geojson_to_ee(gj)


In [ ]:

# === Cargar zonas y asegurar zone_id ===
fc_kml = ee_fc_from_local_kml(KML_LOCAL_PATH)
fc_kmz = ee_fc_from_local_kmz(KMZ_LOCAL_PATH)

def ensure_zone_id(fc: ee.FeatureCollection, prefix: str) -> ee.FeatureCollection:
    lst = fc.toList(fc.size())
    def _mk(i):
        f = ee.Feature(lst.get(i))
        zid = ee.Algorithms.If(f.propertyNames().contains(ZONE_KEY),
                               f.get(ZONE_KEY),
                               ee.String(prefix).cat("-").cat(ee.Number(i).format("%04d")))
        return f.set(ZONE_KEY, zid)
    idx = ee.List.sequence(0, fc.size().subtract(1))
    return ee.FeatureCollection(idx.map(_mk))

fc_kml = ensure_zone_id(fc_kml, "TVCHIAPAS")
fc_kmz = ensure_zone_id(fc_kmz, "OSCAR")
ZONES_FC = fc_kml.merge(fc_kmz)

def dissolve_by_key(fc: ee.FeatureCollection, key: str) -> ee.FeatureCollection:
    keys = fc.aggregate_array(key).distinct()
    def _diss(k):
        k = ee.String(k)
        geom = fc.filter(ee.Filter.eq(key, k)).geometry().dissolve(30)
        return ee.Feature(geom, {key: k})
    return ee.FeatureCollection(keys.map(_diss))

ZONES = dissolve_by_key(ZONES_FC, ZONE_KEY)
AOI = ZONES.geometry()
center = AOI.centroid().coordinates().getInfo()
print("✅ Zonas cargadas. Centro AOI:", center)


## Sentinel‑2 SR → NDVI y NDWI (QA60)

In [ ]:

def mask_s2_sr(image):
    qa = image.select('QA60')
    cloudBitMask  = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return (image.updateMask(mask)
                 .divide(10000)
                 .select(['B2','B3','B4','B8','B11','B12','QA60'])
                 .copyProperties(image, image.propertyNames()))

s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
      .filterBounds(AOI)
      .filterDate(DATE_START, DATE_END)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
      .map(mask_s2_sr))

s2_med = s2.median()
NDVI = s2_med.normalizedDifference(['B8','B4']).rename('NDVI')
NDWI = s2_med.normalizedDifference(['B8','B11']).rename('NDWI')

ndvi_vis = {'min': -0.2, 'max': 0.9, 'palette': ['#a50026','#f46d43','#fee08b','#a6d96a','#1a9850']}
ndwi_vis = {'min': -0.2, 'max': 0.8, 'palette': ['#7f3b08','#fdb863','#e6f598','#67a9cf','#2166ac']}
true_vis = {'bands': ['B4','B3','B2'], 'min': 0.0, 'max': 0.3}

m = leafmap.Map(center=[center[1], center[0]], zoom=13, tiles="CartoDB.Positron")
m.add_basemap("Esri.WorldImagery")
m.addLayer(s2_med, true_vis, "Sentinel-2 True Color")
m.addLayer(NDVI, ndvi_vis, "NDVI (S2 SR)")
m.addLayer(NDWI, ndwi_vis, "NDWI (S2 SR)")
m.addLayer(ZONES.style(color="black", fillColor="00000000", width=2), {}, "ZONAS")
m


## Estadísticos zonales (NDVI/NDWI)

In [ ]:

reducer = (ee.Reducer.mean()
           .combine(ee.Reducer.median(), sharedInputs=True)
           .combine(ee.Reducer.minMax(), sharedInputs=True)
           .combine(ee.Reducer.stdDev(), sharedInputs=True))

stats_ndvi = NDVI.reduceRegions(ZONES, reducer, scale=10).map(
    lambda f: f.set("var", "NDVI", "start", DATE_START.format(), "end", DATE_END.format())
)
stats_ndwi = NDWI.reduceRegions(ZONES, reducer, scale=10).map(
    lambda f: f.set("var", "NDWI", "start", DATE_START.format(), "end", DATE_END.format())
)
zone_stats = stats_ndvi.merge(stats_ndwi)
print("✅ Tabla zonal features:", zone_stats.size().getInfo())

# (Opcional) CSV local rápido
try:
    df = geemap.ee_to_pandas(zone_stats)
    out_csv = "/mnt/data/AlphaEarth_zone_stats_ndvi_ndwi.csv"
    df.to_csv(out_csv, index=False, encoding="utf-8")
    print("💾 CSV local:", out_csv)
except Exception as e:
    print("ℹ️ ee_to_pandas puede requerir permisos; si falla, usa Export.table.toDrive más abajo.")


## Embeddings (Annual, 64D): PCA RGB, Similaridad, Cambio YoY

In [ ]:

emb_ic = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
          .filterDate(ee.Date.fromYMD(YEAR,1,1), ee.Date.fromYMD(YEAR+1,1,1))
          .filterBounds(AOI))
emb_img = emb_ic.mosaic()

arr = emb_img.toArray()
pca = arr.reduce(ee.Reducer.principalComponents(3))
pca_bands = pca.arrayProject([0]).arrayFlatten([['PC1','PC2','PC3']])

ref_vec = emb_img.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=AOI.centroid().buffer(1000),
    scale=10,
    maxPixels=1e9
)
similarity = emb_img.multiply(ee.Image().rename(emb_img.bandNames()).add(ref_vec)).reduce(ee.Reducer.sum())

emb_prev = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
            .filterDate(ee.Date.fromYMD(YEAR-1,1,1), ee.Date.fromYMD(YEAR,1,1))
            .filterBounds(AOI)).mosaic()
change_sim = emb_prev.multiply(emb_img).reduce(ee.Reducer.sum())

m2 = leafmap.Map(center=[center[1], center[0]], zoom=13, tiles="CartoDB.Positron")
m2.addLayer(pca_bands.clip(AOI), {'min': -2, 'max': 2}, f'Embeddings PCA RGB {YEAR}')
m2.addLayer(similarity.clip(AOI), {'min': 0, 'max': 1, 'palette': ['white','blue']}, f'Similaridad {YEAR}')
m2.addLayer(change_sim.clip(AOI), {'min': 0, 'max': 1, 'palette': ['white','black']}, f'Cambio YoY {YEAR-1}->{YEAR}')
m2.addLayer(ZONES.style(color="black", fillColor="00000000", width=2), {}, "ZONAS")
m2


## Exportes opcionales (Drive)

In [ ]:

# === Export.table.toDrive (zonal stats) ===
# task_tbl = ee.batch.Export.table.toDrive(
#     collection=zone_stats,
#     description=f"AlphaEarth_zone_stats_{YEAR}",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"zone_stats_{YEAR}",
#     fileFormat="CSV"
# )
# task_tbl.start()

# === Export.image.toDrive (NDVI/NDWI clip) ===
# task_ndvi = ee.batch.Export.image.toDrive(
#     image=NDVI.clip(ZONES.geometry()),
#     description=f"AlphaEarth_NDVI_clip_{YEAR}",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"ndvi_clip_{YEAR}",
#     scale=10,
#     region=ZONES.geometry(),
#     maxPixels=1e13
# )
# task_ndvi.start()

# task_ndwi = ee.batch.Export.image.toDrive(
#     image=NDWI.clip(ZONES.geometry()),
#     description=f"AlphaEarth_NDWI_clip_{YEAR}",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"ndwi_clip_{YEAR}",
#     scale=10,
#     region=ZONES.geometry(),
#     maxPixels=1e13
# )
# task_ndwi.start()

print("ℹ️ Exports listos para descomentar si deseas enviar a Drive.")
